In [1]:
import pandas as pd

In [8]:
df = pd.read_excel('../ini_data/ini_data_processed.xlsx')
df.columns = ['name', 'delivery_date', 'order_date', 'delivery_period', 'scheduled_delivery_period', 'region', 'quantity', 'price', 'pmt_term', 'non_scheduled', 'supplier', 'order_date_est', 'order_date_factual', 'name2', 'group']

In [9]:
df.head()

,name,delivery_date,order_date,delivery_period,scheduled_delivery_period,region,quantity,price,pmt_term,non_scheduled,supplier,order_date_est,order_date_factual,name2,group
0,ШКИВ К МПЭ-350-900 3536.19.00.200,2017-04-10,2015-06-09,671.0,210.0,Республика Карелия,1,3.839490e+04,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 30 дней с да...,NaN,4011,2015-06-09,1.0,шкив к мпэ-350-900 3536-19-00-200,шкив
1,СТРЕЛА 1085.05.00СБ,2015-11-10,2015-08-03,99.0,180.0,Вологодская область,1,2.050955e+06,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699,2015-08-03,1.0,стрела 1085-05-00сб,стрела
2,УСТРОЙСТВО КУЭ-3 04УХЛ2,2015-10-27,2015-08-03,85.0,150.0,Вологодская область,1,1.337580e+06,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699,2015-08-03,1.0,устройство куэ-3 04ухл2,NaN
3,ВТУЛКА 1080.33.70,2015-08-31,2015-08-03,28.0,30.0,Вологодская область,6,3.439490e+03,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699,2015-08-03,1.0,втулка 1080-33-70,втулка
4,ЗУБ КОВША 1085.52.05-1СБ НАПЛАВКА,2015-08-31,2015-08-03,28.0,75.0,Вологодская область,30,6.496815e+03,ПОСЛЕДУЮЩАЯ ОПЛАТА 100% в течение 60 дней с да...,NaN,3699,2015-08-03,1.0,зуб ковша 1085-52-05-1сб наплавка,зуб


In [31]:
df_gr = df.groupby('name2').count()['name'].sort_values(ascending=False)
df_gr_flt = df_gr[df_gr > 10]
df_gr_flt

name2
зуб 1085-52-05-1               131
колесо 3519-05-02-006           71
вал 3536-11-01-002              57
вал 3519-05-02-083              52
коромысло 3519-21-00-023        48
                              ... 
засов днища 3532-01-02-007      11
рычаг 3711-01-02-050            11
шестерня 3572-05-10-002         11
шпилька 3546-03-04-003          11
ось напорная 3546-03-04-002     11
Name: name, Length: 75, dtype: int64

In [32]:
df_gr_gr = df.groupby('group').count()['name'].sort_values(ascending=False)
df_gr_gr[df_gr_gr > 10].mean()

51.43283582089552